Preparación de datos

In [1]:
# Importar bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
import timeit

In [2]:
# Cargar los datos con manejo de excepciones
try:
    car_data = pd.read_csv('files/datasets/input/car_data.csv')
    print("Los datos se han cargado correctamente.")
except FileNotFoundError:
    print("El archivo no se encontró. Por favor, verifique la ruta del archivo.")

Los datos se han cargado correctamente.


In [3]:
# Muestra inforación del DataFrame
print(car_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
# Manejo de valores faltantes
car_data.dropna(inplace=True)

# Muestra las primeras columnas del DataFrame
display(car_data.head())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
5,04/04/2016 17:36,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,04/04/2016 00:00,0,33775,06/04/2016 19:17
6,01/04/2016 20:48,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,01/04/2016 00:00,0,67112,05/04/2016 18:18
7,21/03/2016 18:54,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,21/03/2016 00:00,0,19348,25/03/2016 16:47


In [5]:
# Tomar una muestra aleatoria del 20% de los datos para reducir el tamaño del conjunto de entrenamiento
car_data_sample = car_data.sample(frac=0.2, random_state=123)

In [6]:
# Identificar características categóricas y numéricas
categorical_cols = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
numeric_cols = ['RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth']

In [7]:
# Dividir los datos en características y variable objetivo
X = car_data.drop('Price', axis=1)
y = car_data['Price']

In [8]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

In [9]:
# Preprocesamiento de datos
numeric_transformer = SimpleImputer(strategy='mean')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [10]:
# Explora estadísticas adicionales del conjunto de datos
print(car_data.describe())

               Price  RegistrationYear          Power        Mileage  \
count  245814.000000     245814.000000  245814.000000  245814.000000   
mean     5125.346717       2002.918699     119.970884  127296.716216   
std      4717.948673          6.163765     139.387116   37078.820368   
min         0.000000       1910.000000       0.000000    5000.000000   
25%      1499.000000       1999.000000      75.000000  125000.000000   
50%      3500.000000       2003.000000     110.000000  150000.000000   
75%      7500.000000       2007.000000     150.000000  150000.000000   
max     20000.000000       2018.000000   20000.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      245814.000000          245814.0  245814.000000  
mean            6.179701               0.0   51463.186002  
std             3.479519               0.0   25838.058847  
min             0.000000               0.0    1067.000000  
25%             3.000000               0.0   30966.

Entrenamiento del modelo

In [11]:
# Definir modelos con hiperparámetros a ajustar
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'LightGBM': lgb.LGBMRegressor(),
    'CatBoost': CatBoostRegressor(verbose=0), 
    'XGBoost': xgb.XGBRegressor()  
}

In [12]:
# Definir los hiperparámetros a ajustar para cada modelo
param_grid = {
    'Linear Regression': {
        'model__fit_intercept': [True, False]
    },
        'Random Forest': {
        'model__n_estimators': [50, 100],
        'model__max_depth': [5, 10],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2]
    },
    'LightGBM': {
        'model__n_estimators': [50, 100],
        'model__max_depth': [5, 10],
        'model__learning_rate': [0.01, 0.1]
    },
    'CatBoost': {
        'model__iterations': [50, 100],
        'model__depth': [5, 10]
    },
    'XGBoost': {
        'model__n_estimators': [50, 100],
        'model__max_depth': [5, 10],
        'model__learning_rate': [0.01, 0.1]
    }
}

In [13]:
# Entrenar y evaluar modelos optimizados
for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    print(f"Entrenando y evaluando el modelo {name}")
    start_time = timeit.default_timer()
    
    # Realizar búsqueda de hiperparámetros mediante GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid.get(name), cv=3, n_jobs=-1, scoring='neg_root_mean_squared_error')
    grid_search.fit(X_train, y_train)
    
    elapsed_time = timeit.default_timer() - start_time
    print(f'Tiempo de entrenamiento: {elapsed_time:.2f} segundos')
    
    # Obtener el mejor modelo
    best_model = grid_search.best_estimator_
    
    # Evaluar el modelo en el conjunto de prueba
    start_time = timeit.default_timer()
    y_pred = best_model.predict(X_test)
    elapsed_time = timeit.default_timer() - start_time
    
    # Calcular el RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    print(f'Tiempo de predicción: {elapsed_time:.2f} segundos')
    print(f'{name} RMSE: {rmse}\n')

Entrenando y evaluando el modelo Linear Regression


Tiempo de entrenamiento: 7.25 segundos
Tiempo de predicción: 0.09 segundos
Linear Regression RMSE: 2986.4195790070366

Entrenando y evaluando el modelo Random Forest


C:\Users\lozano.o\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Tiempo de entrenamiento: 2378.13 segundos


C:\Users\lozano.o\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

C:\Users\lozano.o\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Tiempo de predicción: 0.33 segundos
LightGBM RMSE: 1732.8388605735365

Entrenando y evaluando el modelo CatBoost
Tiempo de entrenamiento: 42.16 segundos


C:\Users\lozano.o\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Tiempo de predicción: 0.36 segundos
CatBoost RMSE: 1627.3635177635906

Entrenando y evaluando el modelo XGBoost
Tiempo de entrenamiento: 39.99 segundos
Tiempo de predicción: 0.25 segundos
XGBoost RMSE: 1623.3818343548207



C:\Users\lozano.o\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
# Análisis adicional
for name, model in models.items():
    if name == 'Linear Regression':
        print("Este es un modelo de regresión lineal.")
    elif name == 'Random Forest':
        print("Este es un modelo de Bosque Aleatorio.")
    elif name == 'LightGBM':
        print("Este es un modelo de LightGBM (Gradient Boosting).")
    elif name == 'CatBoost':
        print("Este es un modelo de CatBoost (Gradient Boosting).")
    elif name == 'XGBoost':
        print("Este es un modelo de XGBoost (Gradient Boosting).")
    print("\n")

Este es un modelo de regresión lineal.


Este es un modelo de Bosque Aleatorio.


Este es un modelo de LightGBM (Gradient Boosting).


Este es un modelo de CatBoost (Gradient Boosting).


Este es un modelo de XGBoost (Gradient Boosting).


